In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/cnn2d_muse_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_gru_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_muse_adj_1_feat_de_fr.pkl
(159571, 6) (153164, 6)
file path ../features/cudnn_gru_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cudnn_gru_glove_1_sample_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/other_feat.pkl
(159571, 16) (153164, 16)
file path ../features/tfidf_feat1.pkl
(159571, 30) (153164, 30)
file path ../features/tfidf_feat2.pkl
(159571, 30) (153164, 30)
(159571, 142)
(159571, 142)


In [2]:
train[list_classes].describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
def aug_data_ratio(x,y,ratio=0.2):
    print(x.shape)
    neg_index = np.where(y[:,0]==0)[0]
    print(neg_index)
    data_cnt = len(neg_index)
    add_cnt = int(data_cnt*ratio)
    add_index = neg_index[:add_cnt]
    add_x = np.concatenate([x,x[add_index]])
    add_y = np.concatenate([y,y[add_index]])
    print(add_x.shape,data_cnt)
    return add_x,add_y

def del_data_ratio(x,y,ratio=0.8):
    print(x.shape)
    pos_index = np.where(y[:,0]==1)[0]
    neg_index = np.where(y[:,0]==0)[0]
    print(pos_index)
    data_cnt = len(pos_index)
    add_cnt = int(data_cnt*ratio)
    add_index = pos_index[:add_cnt]
    add_x = np.concatenate([x[add_index],x[neg_index]])
    add_y = np.concatenate([y[add_index],y[neg_index]])
    print(add_x.shape,data_cnt)
    return add_x,add_y

# add neg data
#train_x,train_y = aug_data_ratio(train_x,train_y)
# del pos data
train_x,train_y = del_data_ratio(train_x,train_y)

(159571, 142)
[     6     12     16 ..., 159541 159546 159554]
(156512, 142) 15294


In [4]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cache_test_pred = np.zeros((153164,6))
    single_best = 100
    single_best_pred = None
    all_train_loss_l,all_val_loss_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        d_test = xgb.DMatrix(test_x)
        
        # share params
        params = {
                'colsample_bytree': 0.9,
                'subsample': 0.9,
                'eta': 0.05,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight':0.9,
                'colsample_bylevel':0.9,
                'colsample_bytree':0.9,
            
                }
        
        # train for each class
        for i in range(6):
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]

            model = xgb.train(params, d_train, 1000, watchlist,
                              early_stopping_rounds=50,
                              verbose_eval=2000)
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict(d_train))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict(d_valid))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict(d_test)
            
            test_pred[:,i] += curr_test_pred
            cache_test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # save best one fold result
        if val_loss_l < single_best:
            single_best = val_loss_l
            single_best_pred = cache_test_pred
            print('new single best')
        
        cache_test_pred = np.zeros((153164,6))
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred, single_best_pred

print('done')

done


In [5]:
xgb_res,b = simple_ens('xgb',k=3)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_sample_csv_fold3.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')


[0]	train-logloss:0.648654	valid-logloss:0.648733
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[410]	train-logloss:0.061667	valid-logloss:0.071509

0
0.0605331505449 0.071568105252
[0]	train-logloss:0.645432	valid-logloss:0.645389
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[176]	train-logloss:0.014776	valid-logloss:0.016605

1
0.0138355341536 0.0166496193912
[0]	train-logloss:0.646551	valid-logloss:0.646595
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[260]	train-logloss:0.031612	valid-logloss:0.036709

2
0.0304812294588 0.0367199800867
[0]	train-logloss:0.644694	valid-logloss:0.644733
Multiple 

In [ ]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res/1.2
sample_submission.to_csv("../results/xgb_ens_sample_csv_fold3_div2.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.831555      0.234100  0.816523  0.105206  0.776196   
1  0000247867823ef7  0.000082      0.000025  0.000065  0.000018  0.000059   
2  00013b17ad220c46  0.000541      0.000031  0.000278  0.000029  0.000283   
3  00017563c3f7919a  0.000093      0.000028  0.000095  0.000023  0.000090   
4  00017695ad8997eb  0.002165      0.000023  0.000533  0.000035  0.000189   

   identity_hate  
0       0.434721  
1       0.000038  
2       0.000206  
3       0.000053  
4       0.000132  
save done


In [ ]:
xgb_res,b = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res/1.2
sample_submission.to_csv("../results/xgb_ens_sample_csv_fold5_div2.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')


[0]	train-logloss:0.648585	valid-logloss:0.648716
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[630]	train-logloss:0.058642	valid-logloss:0.071513

0
0.0577842140128 0.0715380932188
[0]	train-logloss:0.645414	valid-logloss:0.645415
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[283]	train-logloss:0.013211	valid-logloss:0.016794

1
0.0124955328822 0.0168663720067
[0]	train-logloss:0.646534	valid-logloss:0.646599
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[292]	train-logloss:0.031937	valid-logloss:0.035503

2
0.0310307665851 0.0355704225628
[0]	train-logloss:0.644699	valid-logloss:0.644724
Multiple